# **Lab 3: Path Planning**

## What is Path Planning?
Path planning, also known as motion planning, is a computational process that determines a viable path for an autonomous agent (e.g., a robot) to move from a starting point to a destination without colliding with obstacles. It is a fundamental aspect of robotics, enabling robots to operate autonomously in diverse and dynamic environments.

## Applications in Robotics
Path planning is crucial in various robotic applications, including but not limited to:

- Autonomous Vehicles: Navigating roads, avoiding obstacles, and reaching destinations efficiently.
- Industrial Automation: Moving robotic arms and autonomous mobile robots (AMRs) within factories.
- Service Robots: Assisting in tasks within homes, hospitals, and public spaces.
- Exploration Robots: Navigating unknown terrains in space missions or underwater exploration.

## Challenges in Path Planning

Path planning involves addressing several challenges to ensure efficient and safe navigation:

- Dynamic Environments: Environments that change over time require adaptive planning.
- High-Dimensional Spaces: Robots with many degrees of freedom operate in complex spaces.
- Real-Time Constraints: Time-sensitive applications demand swift computation.
- Optimization Goals: Balancing multiple objectives like shortest path, energy efficiency, and safety.

---

## **Packages Needed**

In [1]:
import pygame
import math
import sys
import random
from collections import deque
from queue import PriorityQueue
import heapq

pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


## **Environment Setup**
### Grid-Based Representation
To simplify the path planning problem, we represent the environment as a 2D grid where each cell signifies a discrete position in space. This approach discretizes continuous spaces, making them more manageable for algorithmic processing.

- Free Cell (White): Indicates a traversable space.
- Obstacle (Black): Represents an impassable barrier.
- Start Position (Orange): The cell where the robot begins.
- Goal Position (Tirqoise): The target cell the robot needs to reach.
- Path (Purple): The path found by the algorithm.

#### Interactive Grid Interface
We'll use Pygame to create an interactive grid where users can:

- Set Start and Goal Points: Click to define the start and goal positions.
- Place Obstacles: Draw obstacles by clicking and dragging the mouse.
- Run Algorithms: Select and execute the desired path planning algorithm to visualize the pathfinding process.

In [2]:
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("Path Planning Algorithm Visualization")

# Colors for the grid visualization
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)


In [3]:
# Spot class to handle the grid positions and status
class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == RED

    def is_open(self):
        return self.color == GREEN

    def is_barrier(self):
        return self.color == BLACK

    def is_start(self):
        return self.color == ORANGE

    def is_end(self):
        return self.color == TURQUOISE

    def reset(self):
        self.color = WHITE

    def make_start(self):
        self.color = ORANGE

    def make_closed(self):
        self.color = RED

    def make_open(self):
        self.color = GREEN

    def make_barrier(self):
        self.color = BLACK

    def make_end(self):
        self.color = TURQUOISE

    def make_path(self):
        self.color = PURPLE

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors = []
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # Down
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # Up
            self.neighbors.append(grid[self.row - 1][self.col])

        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # Right
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # Left
            self.neighbors.append(grid[self.row][self.col - 1])

    def __lt__(self, other):
        return False

In [4]:
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, j, gap, rows)
            grid[i].append(spot)
    return grid

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))

def draw(win, grid, rows, width):
    win.fill(WHITE)
    for row in grid:
        for spot in row:
            spot.draw(win)
    draw_grid(win, rows, width)
    pygame.display.update()

def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()


---

## **Breadth-First Search (BFS)**
### Algorithm Overview
Breadth-First Search (BFS) is a fundamental graph traversal algorithm that explores vertices in the order of their distance from the source node, ensuring that all nodes at a given depth are visited before moving to the next level. In the context of path planning:

- Traversal Order: Expands equally in all directions from the start position.
- Guarantees: Finds the shortest path in an unweighted grid.
### Theoretical Analysis
Time Complexity: O(V + E)

- V: Number of vertices (cells in the grid).
- E: Number of edges (connections between cells).
- In a grid, each cell has up to four neighbors (up, down, left, right), so E ≈ 4V.

Space Complexity: O(V)

Due to the storage of the queue and visited set.
### Advantages:

- Shortest Path: Guarantees the shortest path in terms of the number of steps in an unweighted grid.
- Simplicity: Easy to implement and understand.
### Disadvantages:

- Memory Intensive: Can consume a lot of memory for large grids.
- No Heuristic: Does not prioritize paths leading closer to the goal, potentially exploring unnecessary areas.

In [5]:
# Breadth-First Search Algorithm
def bfs(draw, grid, start, end):
    queue = deque([start])
    came_from = {}
    visited = {start}

    while queue:
        current = queue.popleft()

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            if neighbor not in visited and not neighbor.is_barrier():
                queue.append(neighbor)
                visited.add(neighbor)
                came_from[neighbor] = current
                neighbor.make_open()

        draw()
        if current != start:
            current.make_closed()

    return False

### Explanation:

1. Initialization:

    - Queue: Initialized with the start position.
    - Visited Set: Tracks visited cells to prevent revisiting.
    - Parent Dictionary: Stores the path by keeping track of each cell's predecessor.
2. Traversal:

    - Dequeue the first cell.
    - Check if it's the goal; if so, terminate.
    - Explore all valid neighbors (up, down, left, right) that are free and not visited.
    - Enqueue valid neighbors and update their parents.
3. Path Reconstruction:

    - Starting from the goal, backtrack using the parent dictionary to reconstruct the path to the start.
    - If the goal is unreachable, return None.

---

## **Depth-First Search (DFS)**
### Algorithm Overview
Depth-First Search (DFS) is a graph traversal algorithm that explores as far as possible along each branch before backtracking. Unlike BFS, DFS dives deep into one path before exploring others.

### In the context of path planning:

- Traversal Order: Explores one direction fully before backtracking.
- Guarantees: Does not guarantee the shortest path.
### Theoretical Analysis
- Time Complexity: O(V + E)

- Space Complexity: O(V)

### Advantages:

- Low Memory Usage: Uses less memory compared to BFS as it doesn't store all nodes at a level.
- Simplicity: Straightforward to implement, especially using recursion.
### Disadvantages:

- No Shortest Path Guarantee: May find a longer path compared to BFS.
- Potential for Deep Recursion: Can lead to stack overflow in deep or infinite graphs.

In [6]:
# Depth-First Search Algorithm
def dfs(draw, grid, start, end):
    stack = [start]
    came_from = {}
    visited = {start}

    while stack:
        current = stack.pop()

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            if neighbor not in visited and not neighbor.is_barrier():
                stack.append(neighbor)
                visited.add(neighbor)
                came_from[neighbor] = current
                neighbor.make_open()

        draw()
        if current != start:
            current.make_closed()

    return False

### Explanation:

1. Initialization:

    - Stack: Initialized with the start position.
    - Visited Set: Tracks visited cells.
    - Parent Dictionary: Keeps track of the path.
2. Traversal:

    - Pop the last cell from the stack.
    - Check if it's the goal; if so, terminate.
    - Explore all valid neighbors and push them onto the stack.
    - Mark neighbors as visited and record their parent.
3. Path Reconstruction:

    - Similar to BFS, backtrack from the goal to the start using the parent dictionary.
    - If the goal is unreachable, return None.

---

## **Dijkstra's Algorithm**
### Algorithm Overview
Dijkstra's Algorithm is a graph search algorithm that finds the shortest path between nodes in a graph with non-negative edge weights. It systematically explores the graph, always expanding the node with the lowest cumulative cost from the start.

### In the context of path planning:

- Traversal Order: Prioritizes nodes with the smallest distance from the start.
- Guarantees: Finds the shortest path in weighted grids.
### Theoretical Analysis
Time Complexity: O((V + E) log V)

- V: Number of vertices.
- E: Number of edges.
- Log V: Due to the priority queue operations.
- Space Complexity: O(V)

### Advantages:

- Optimality: Guarantees the shortest path in weighted graphs.
- Flexibility: Can handle varying edge costs.
### Disadvantages:

- Performance: Slower compared to BFS and DFS in unweighted grids.
- Memory Usage: Requires storage of all nodes in the priority queue.

In [7]:
# Dijkstra's Algorithm
def dijkstra(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    open_set_hash = {start}

    while not open_set.empty():
        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((g_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()
        if current != start:
            current.make_closed()

    return False


### Explanation:

1. Initialization:

    - Priority Queue (Heap): Stores cells with their cumulative costs, prioritized by lowest cost.
    - Visited Set: Tracks visited cells to prevent redundant processing.
    - Parent Dictionary: Records the path.
2. Traversal:

    - Pop the cell with the lowest cumulative cost.
    - If it's the goal, terminate.
    - Explore all valid neighbors.
    - Calculate the new cost to reach each neighbor.
    - If the new cost is lower, update the cost and enqueue the neighbor.
3. Path Reconstruction:

    - Backtrack from the goal using the parent dictionary to form the shortest path.
    - Return None if no path exists.

---

## **A\* (A-Star) Algorithm**
### Algorithm Overview
A* (A-Star) Algorithm is an informed search algorithm that enhances Dijkstra's by incorporating heuristics to prioritize nodes that are likely closer to the goal. This makes A* more efficient by reducing the search space. <br>
![A*](figures/A-star.png)


### In the context of path planning:

- Traversal Order: Prioritizes nodes based on the sum of the cost to reach them and an estimated cost to the goal.
- Guarantees: Finds the shortest path if the heuristic is admissible (never overestimates the true cost).
### Heuristic Functions
- A heuristic function estimates the cost from a node to the goal. The choice of heuristic significantly impacts A*'s performance.
#### Common Heuristics:

##### Manhattan Distance:
- measures the distance between two points by only allowing movement along axes at right angles. Imagine navigating a grid-like city (like Manhattan), where you can only move horizontally or vertically. <br>
![manhattan](figures/manhattan.png)

##### Euclidean Distance:
- most commonly used distance metric, representing the "straight-line" distance between two points in Euclidean space. It's akin to measuring the shortest path between two points using a ruler. <br>
![euclidean](figures/euclidean.png)

##### Chebyshev Distance:
- measures the greatest of the differences along any coordinate dimension. It represents the minimum number of moves a king would take to move from one square to another on a chessboard, allowing movement in any direction (including diagonally). <br>
![chebychev](figures/chebyshev.png)



Admissibility: A heuristic is admissible if it never overestimates the true cost to reach the goal. This ensures that A* finds the optimal path.<br>

![all](figures/all.jpg)


### Theoretical Analysis
    - Time Complexity: O(E) in the worst case, but typically much better due to heuristics.

    - Space Complexity: O(V)
### Advantages:
    - Efficiency: Heuristics guide the search, reducing the number of explored nodes.
    - Optimality: Guarantees the shortest path with admissible heuristics.

### Disadvantages:
    - Heuristic Dependency: Performance heavily relies on the chosen heuristic.
    - Memory Usage: Can still consume significant memory for large grids.

In [8]:
def heuristic(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

def a_star(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    f_score = {spot: float("inf") for row in grid for spot in row}
    f_score[start] = heuristic(start.get_pos(), end.get_pos())
    open_set_hash = {start}

    while not open_set.empty():
        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + heuristic(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()

        if current != start:
            current.make_closed()

    return False

### Explanation:

#### Heuristic Function:

We use the Manhattan distance as it's suitable for grid-based environments without diagonal movements.
1. Initialization:

    - Priority Queue (Heap): Stores cells with their priority (cumulative cost + heuristic).
    - Visited Set: Tracks visited cells.
    - Parent Dictionary: Records the path.
2. Traversal:

    - Pop the cell with the lowest priority.
    - If it's the goal, terminate.
    - Explore all valid neighbors.
    - Calculate the new cost to reach each neighbor.
    - Update costs and enqueue neighbors with updated priorities.

3. Path Reconstruction:

    - Similar to BFS and Dijkstra's, backtrack from the goal using the parent dictionary.
    - Return None if no path exists.

---

### Main Function

In [9]:
pygame.init()

def main(win, width):
    ROWS = 20
    grid = make_grid(ROWS, width)

    start = None
    end = None

    run = True
    while run:
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # LEFT CLICK
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                if not start and spot != end:
                    start = spot
                    start.make_start()

                elif not end and spot != start:
                    end = spot
                    end.make_end()

                elif spot != end and spot != start:
                    spot.make_barrier()

            elif pygame.mouse.get_pressed()[2]:  # RIGHT CLICK
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)

                    # Uncomment one of these to run the respective algorithm

                    bfs(lambda: draw(win, grid, ROWS, width), grid, start, end)
                    # dfs(lambda: draw(win, grid, ROWS, width), grid, start, end)
                    # dijkstra(lambda: draw(win, grid, ROWS, width), grid, start, end)
                    # a_star(lambda: draw(win, grid, ROWS, width), grid, start, end)

    pygame.quit()

main(WIN, WIDTH)


<hr style="height:5px; border:none; background-color:#ffffff; border-radius:1px;">


# Robot Navigation
We now will simulate a robot navigating a 2D environment populated with obstacles. The robot is equipped with multiple sensors that detect obstacles within a certain range and field of view. Based on sensor data, the robot employs autonomous behavior to avoid collisions and navigate the environment efficiently. Effective navigation combines various disciplines, including:

- Perception: Sensing the environment using sensors (e.g., cameras, LIDAR, ultrasonic sensors).
- Localization: Determining the robot's position within the environment.
- Path Planning: Computing a feasible and efficient route from the current position to the target destination.
- Control: Executing movements to follow the planned path accurately.

## Key Features:

- Visualization: Uses Pygame to render the simulation window, robot, and obstacles.
- Autonomous Navigation: Robot moves forward, senses obstacles, and adjusts its path to avoid collisions.
- Sensor Simulation: Implements ray casting to simulate sensor readings detecting obstacle distances.
- Collision Handling: Detects and responds to collisions by altering the robot's direction.

---

### Constants and Configuration
#### Purpose:

- Window Dimensions (WIDTH, HEIGHT): Defines the size of the simulation window.
- Colors: Specifies RGB tuples for various elements (background, robot, sensors, obstacles).
- Robot Properties:
    - ROBOT_RADIUS: Size of the robot for rendering and collision detection.
    - SENSOR_RANGE: Maximum distance each sensor can detect obstacles.
    - SENSOR_FOV: Total field of view covered by all sensors (in degrees).
    - SENSOR_COUNT: Number of individual sensors distributed within the field of view.
- Obstacles:
    - OBSTACLE_COLOR: Color used to render obstacles.
    - OBSTACLES: List to store obstacle objects (Pygame rectangles).

In [10]:
WIDTH, HEIGHT = 800, 600

In [11]:
WHITE = (255, 255, 255)
GRAY = (50, 50, 50)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

In [12]:
ROBOT_RADIUS = 15
SENSOR_RANGE = 100
SENSOR_FOV = 180  # Field of view in degrees
SENSOR_COUNT = 9  # Number of sensors

OBSTACLE_COLOR = GRAY

---

### Screen Borders
#### Functionality:

- Purpose: Creates four rectangular borders around the simulation window to prevent the robot from moving out of bounds.
- Implementation:
    - Thickness: Each border has a fixed thickness (border_thickness = 5 pixels).
    - Placement: Adds top, bottom, left, and right borders as pygame.Rect objects to the OBSTACLES list.

In [13]:
OBSTACLES = [] #List to store in it the generated obstacles

In [14]:
def create_screen_borders():
    border_thickness = 5
    OBSTACLES.append(pygame.Rect(0, 0, WIDTH, border_thickness))
    OBSTACLES.append(pygame.Rect(0, HEIGHT - border_thickness, WIDTH, border_thickness))
    OBSTACLES.append(pygame.Rect(0, 0, border_thickness, HEIGHT))
    OBSTACLES.append(pygame.Rect(WIDTH - border_thickness, 0, border_thickness, HEIGHT))

---

### Random Obstacle Generation
#### Functionality:

- Purpose: Generates a specified number of random rectangular obstacles within the simulation window.
- Implementation:
    - Margin: Ensures that obstacles are not placed too close to the window edges (margin = 50 pixels).
    - Dimensions: Each obstacle has a random width and height between 30 and 100 pixels.
    - Placement: Randomly positions each obstacle within the allowable area and adds it to the OBSTACLES list.


In [15]:
def create_obstacles(num_obstacles):
    margin = 50  # Ensure obstacles are not too close to the edges
    for _ in range(num_obstacles):
        x = random.randint(margin, WIDTH - margin - 50)
        y = random.randint(margin, HEIGHT - margin - 50)
        width = random.randint(30, 100)
        height = random.randint(30, 100)
        rect = pygame.Rect(x, y, width, height)
        OBSTACLES.append(rect)

---

### Robot Class
The Robot class encapsulates all behaviors and properties of the simulated robot.

#### Attributes:

- Position (x, y): Current coordinates of the robot within the simulation window.
- Radius (self.radius): Size of the robot for rendering and collision detection.
- Orientation (self.angle): Current facing direction in degrees, initialized randomly between 0° and 360°.
- Movement Speed (self.speed): Pixels moved per frame when moving forward.
- Turning Speed (self.turn_speed): Degrees the robot turns per frame when adjusting direction.

In [16]:
class Robot:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.radius = ROBOT_RADIUS
        self.angle = random.uniform(0, 360)  # Random initial angle
        self.speed = 2
        self.turn_speed = 5  # Degrees per frame

    def move_forward(self):
        """
        Advances the robot forward in the direction it's currently facing.
        Converts the current angle from degrees to radians.
        Updates the x and y positions based on the cosine and sine of the angle, scaled by the speed.
        """
        rad = math.radians(self.angle)
        self.x += self.speed * math.cos(rad)
        self.y += self.speed * math.sin(rad)

    def turn(self, angle):
        """
        Adjusts the robot's angle by adding the provided angle.
        Ensures the angle remains within the 0°-359° range using modulo operation.
        """
        self.angle += angle
        self.angle %= 360

    def get_sensor_readings(self):
        """
        Simulates sensor data by casting rays at different angles within the robot's field of view.
        Begins at the leftmost angle (self.angle - SENSOR_FOV / 2).
        Determines the angular spacing between consecutive sensors.
        For each sensor, casts a ray at the calculated angle and records the distance to the nearest obstacle.
        """
        readings = []
        start_angle = self.angle - SENSOR_FOV / 2
        angle_increment = SENSOR_FOV / (SENSOR_COUNT - 1)

        for i in range(SENSOR_COUNT):
            sensor_angle = start_angle + i * angle_increment
            distance = self.cast_ray(sensor_angle)
            readings.append(distance)
        return readings

    def cast_ray(self, angle):
        """
        Determines the distance from the robot to the nearest obstacle in a specific direction.
        Calculates the end point of the sensor ray based on the sensor range and angle.
        Iterates through all obstacles to find the closest intersection point with the ray.
        Keeps track of the shortest distance detected by the ray.
        """
        rad = math.radians(angle)
        x_end = self.x + SENSOR_RANGE * math.cos(rad)
        y_end = self.y + SENSOR_RANGE * math.sin(rad)

        shortest_distance = SENSOR_RANGE
        # Check for collisions with obstacles
        for obstacle in OBSTACLES:
            hit, dist = self.ray_intersects_rect((self.x, self.y), (x_end, y_end), obstacle)
            if hit and dist < shortest_distance:
                shortest_distance = dist
        return shortest_distance

    def ray_intersects_rect(self, start, end, rect):
        """
        Determines whether a ray intersects with a given rectangle (obstacle) and calculates the distance to the intersection point if it exists.
        Utilizes Liang-Barsky algorithm for efficient line-clipping and intersection detection.
        Outcome: 
            hit: Boolean indicating whether an intersection occurred.
            distance: Distance from the ray's origin to the intersection point.
        """
        # Ray-Rectangle intersection using Liang-Barsky algorithm.

        x0, y0 = start
        x1, y1 = end

        p = [-(x1 - x0), x1 - x0, -(y1 - y0), y1 - y0]
        q = [x0 - rect.left, rect.right - x0, y0 - rect.top, rect.bottom - y0]

        u1, u2 = 0, 1  # Start and end points of the line segment

        for i in range(4):
            if p[i] == 0:
                if q[i] < 0:
                    return False, None
            else:
                t = q[i] / p[i]
                if p[i] < 0:
                    if t > u2:
                        return False, None
                    elif t > u1:
                        u1 = t
                else:
                    if t < u1:
                        return False, None
                    elif t < u2:
                        u2 = t

        if u2 < u1: #the entering point comes after the exiting point 
            return False, None

        # Calculate intersection point
        xi = x0 + u1 * (x1 - x0)
        yi = y0 + u1 * (y1 - y0)
        distance = math.hypot(xi - x0, yi - y0)
        return True, distance

    def detect_collision(self):
        """
        Checks if the robot has collided with any obstacles.
        Represents the robot as a rectangle based on its current position and radius.
        Uses Pygame's colliderect method to determine if the robot's bounding rectangle overlaps with any obstacle.
        """
        # Robot bounding circle collision with obstacles
        robot_rect = pygame.Rect(self.x - self.radius, self.y - self.radius,
                                 self.radius * 2, self.radius * 2)
        for obstacle in OBSTACLES:
            if robot_rect.colliderect(obstacle):
                return True
        return False

    def update(self):
        self.move_forward()

    def draw(self, surface):
        """
        Renders the robot and its sensors on the simulation window.
        """
        pygame.draw.circle(surface, BLUE, (int(self.x), int(self.y)), self.radius)

        rad = math.radians(self.angle)
        x_end = self.x + self.radius * math.cos(rad)
        y_end = self.y + self.radius * math.sin(rad)
        pygame.draw.line(surface, WHITE, (self.x, self.y), (x_end, y_end), 2)

        start_angle = self.angle - SENSOR_FOV / 2
        angle_increment = SENSOR_FOV / (SENSOR_COUNT - 1)

        for i in range(SENSOR_COUNT):
            sensor_angle = start_angle + i * angle_increment
            rad = math.radians(sensor_angle)
            distance = self.cast_ray(sensor_angle)

            x_end = self.x + distance * math.cos(rad)
            y_end = self.y + distance * math.sin(rad)
            pygame.draw.line(surface, RED, (self.x, self.y), (x_end, y_end), 1)


The p and q arrays are coefficients derived from the parametric line equation and the boundaries of the rectangle. <br>
p = [-(x1-x0), x1-x0, -(y1-y0), y1-y0]<br>
q = [x0−rect.left,rect.right−x0,y0−rect.top,rect.bottom−y0]<br>
- Parameters u1 and u2:

    - u1: Represents the lower bound of the parameter t where the line enters the clipping window.
    - u2: Represents the upper bound of the parameter t where the line exits the clipping window.
    - Both are initialized to 0 and 1, representing the entire line segment.

ooping Through Boundaries:

- The loop iterates over the four boundaries of the rectangle:

    - Left Boundary (i=0):
        - p0 = -(x1-x0)
        - q0 = x0 - rect.left
        - etc.

- Clipping Conditions:

- For each boundary, the algorithm checks whether the ray intersects with the rectangle:
    - Parallel Check (p[i] == 0):
        - If the ray is parallel to the boundary and lies outside the boundary (q[i] < 0), there's no intersection.
    - calculating t = q[i]/p[i]
        - t is a parameter calculating the potential intersecion point within a boundry
    - updating u1 and u2:
        - u1: Ensures that the entering point is the farthest into the window across all boundaries.
        - u2: Ensures that the exiting point is the nearest out of the window across all boundaries.
        - if p[i] < 0 (line entering the clipping window), it implies the line is entering the clipping window
            - update u1 = max(u1, t)
            - if t > u2, no intersection


---

### Obstacle Avoidance Logic

#### Functionality:

- Purpose: Determines how the robot should adjust its movement based on sensor readings to avoid obstacles.
- Implementation:
    - Front Sensors Analysis:
        - Selection: Focuses on sensors directly in front of the robot.
        - Minimum Distance: Finds the closest obstacle detected by the front sensors.
        - Turning Decision: If an obstacle is too close (min_distance < threshold), the robot turns away from the obstacle. The turning strength is proportional to how close the obstacle is.
    - Side Sensors Analysis:
        - Left and Right Sensors: Checks the extreme left and right sensors to detect nearby obstacles.
        - Side Threshold: If an obstacle is detected within the side threshold, the robot adjusts its direction slightly to avoid a side collision.
    - Forward Movement: If no immediate threats are detected, the robot continues moving forward.

In [17]:
def avoid_obstacles(robot, sensor_data):
    threshold = 40  # Increased threshold for better avoidance
    front_sensors = sensor_data[SENSOR_COUNT // 2 - 1:SENSOR_COUNT // 2 + 2]
    min_distance = min(front_sensors)
    min_index = front_sensors.index(min_distance) + SENSOR_COUNT // 2 - 1

    if min_distance < threshold:
        # Obstacle is too close, adjust turning rate based on proximity
        turn_strength = (threshold - min_distance) / threshold
        # Determine turn direction
        if min_index < SENSOR_COUNT // 2:
            robot.turn(robot.turn_speed * turn_strength)
        else:
            robot.turn(-robot.turn_speed * turn_strength)
    else:
        # Check side sensors to avoid getting too close to obstacles
        left_distance = sensor_data[0]
        right_distance = sensor_data[-1]
        side_threshold = 60
        if left_distance < side_threshold:
            robot.turn(robot.turn_speed * 0.5)
        elif right_distance < side_threshold:
            robot.turn(-robot.turn_speed * 0.5)
        # Move forward
        robot.move_forward()

---

### Initialization and Main Loop

In [18]:
OBSTACLES = []

pygame.init()
# Create simulation window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D Robotics Simulation")
clock = pygame.time.Clock()

running = True
robot = Robot(WIDTH // 2, HEIGHT // 2)
create_screen_borders()  # Add screen borders as obstacles
create_obstacles(10)     # Create random obstacles

while running:
    screen.fill(WHITE)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    sensor_data = robot.get_sensor_readings()

    avoid_obstacles(robot, sensor_data)

    robot.update()

    # Collision detection
    if robot.detect_collision():
        # make the robot turn away
        robot.turn(180)
        robot.move_forward()

    for obstacle in OBSTACLES:
        pygame.draw.rect(screen, OBSTACLE_COLOR, obstacle)

    robot.draw(screen)

    pygame.display.flip()
    clock.tick(60) 

pygame.quit()
sys.exit()

SystemExit: 

C:\Users\MoazE\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


---

- Potential Extensions:

    - Enhanced Sensors: Incorporate more sophisticated sensor models or increase sensor count for better environment awareness.
    - Multiple Robots: Introduce additional robots to simulate interactions and more complex navigation scenarios.
    - Dynamic Obstacles: Allow obstacles to move or appear/disappear dynamically to increase simulation complexity.

---